In [ ]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# get original code from https://github.com/xxlong0/Wonder3D.git
%cd /content
!git clone https://github.com/xxlong0/Wonder3D.git
%cd /content/Wonder3D


# install dependencies (from https://github.com/camenduru/Wonder3D-colab.git and some minor personal adjustments)
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/random_states_0.pkl -d /content/Wonder3D/ckpts -o random_states_0.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/scaler.pt -d /content/Wonder3D/ckpts -o scaler.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/scheduler.bin -d /content/Wonder3D/ckpts -o scheduler.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/resolve/main/unet/diffusion_pytorch_model.bin -d /content/Wonder3D/ckpts/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Wonder3D/raw/main/unet/config.json -d /content/Wonder3D/ckpts/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/segment_anything/resolve/main/sam_vit_h_4b8939.pth -d /content/Wonder3D/sam_pt -o sam_vit_h_4b8939.pth

!pip install -q fire diffusers==0.19.3 transformers bitsandbytes accelerate gradio rembg segment_anything
!pip install -q einops omegaconf pytorch-lightning==1.9.5 torch_efficient_distloss nerfacc==0.3.3 PyMCubes trimesh
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl
#!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl
!git clone https://huggingface.co/camenduru/torch_extensions_0_3_3_colab /root/.cache/torch_extensions

In [ ]:
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

In [ ]:
# Load initial Image from drive (remove Background of image)
!cp /content/drive/MyDrive/Images/snowGlobe.png /content/Wonder3D/example_images

In [ ]:
# Alternative: upload image directly
from PIL import Image
from google.colab import files

uploaded = files.upload()

In [ ]:
# Get color and normal from various viewpoints
# change image name accordingly

!accelerate launch --config_file 1gpu.yaml test_mvdiffusion_seq.py \
            --config configs/mvdiffusion-joint-ortho-6views.yaml validation_dataset.root_dir=./example_images \
            validation_dataset.filepaths=['snowGlobe.png'] save_dir=./outputs

In [ ]:
# create a mesh
# for dataset.scene=XXX use the base input name, without .png

%cd ./instant-nsr-pl
!python launch.py --config configs/neuralangelo-ortho-wmask.yaml --gpu 0 --train dataset.root_dir=/content/Wonder3D/outputs/cropsize-192-cfg1.0/snowGlobe dataset.scene=snowGlobe

In [ ]:
# Copy results to drive
!cp -r /content/Wonder3D/outputs/cropsize-192-cfg1.0/snowGlobe /content/drive/MyDrive/snowGlobe
!cp -r /content/Wonder3D/instant-nsr-pl/exp/snowGlobe/@20231215-034305 /content/drive/MyDrive/snowGlobe